# Homework 3 (Tutorial 7)

##MA course in Artificial Intelligence 2022/2023

@author: Filippo Casari

In [ ]:
#!rm -r AI2022MA/
#!git clone https://github.com/UmbertoJr/AI2022MA.git &> /dev/null

In [ ]:
# Imports

#from AI2022MA.IO_manager.io_tsp import TSP_Instance_Creator
# if you are running from your local remove the prefix AI2020 (comment the previous line and uncomment the following line)
from IO_manager.io_tsp import TSP_Instance_Creator

ic = TSP_Instance_Creator("standard", 'eil76.tsp')
ic.print_info()
#ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'ch130.tsp')
ic.print_info()
#ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'd198.tsp')
ic.print_info()
#ic.plot_data()

In [ ]:
ic = TSP_Instance_Creator("standard", 'myTSP_dim10.tsp')
ic.print_info()
ic.plot_data()

In [ ]:

from matplotlib import pyplot as plt
#plt.ion()


def plot_tour(instance, tour, ant):

    plt.figure(figsize=(8, 8))
    plt.grid()
    plt.title(f"Tour Ant # {ant}")
    plt.scatter(instance.points[:, 1], instance.points[:, 2])
    for t in range(len(tour)-1):
        xy1, xy2 = tour[t], tour[t+1]
        plt.plot([instance.points[xy1, 1], instance.points[xy2, 1]], [
                 instance.points[xy1, 2], instance.points[xy2, 2]], color="blue")

        plt.draw()
        plt.pause(0.2)
    # plt.show()


In [ ]:
import numpy as np
import random
from solvers.local_search import twoOpt
from solvers.constructive_algorithms import nn
from solvers.two_opt_with_candidate import twoOpt_with_cl
from threading import Thread
from time import sleep
# nn takes as input the distance matrix and returns
# the tour and the length constructed with nearest neighbor, i.e.   tour, len_t = nn(dist_mat)

# twoOpt takes as input the solution, the actual_len and the distance matrix
# and returns the tour and the length created with 2-opt, i.e.     tour, lent_t = twoOpt(solution, actual_len, dist_mat)


class ACS:
    m = 10
    beta = 2
    alpha = rho = 0.1
    cl = 15
    q0 = 0.98
    stop_after_secs = 0

    @staticmethod
    def take_candidates(j, dist_mat):
        # print(dist_mat[j][1:ACS.cl+1])

        # print(dist_mat[j][1:])
        # print(sorted(dist_mat[j])[1:ACS.cl+1])
        # print(list(np.argsort(dist_mat[j])[1:ACS.cl+1]))

        # da 1 citta vicina a 21 citta', sorting delle citta vicine
        return (np.argsort(dist_mat[j])[1:ACS.cl+1])

    @staticmethod
    def take_other_cities(j, dist_mat):
        return (np.argsort(dist_mat[j])[ACS.cl+1:])

    def __init__(self, instance, q0=None, boost=False, timeStop=180):
        """constructor

        Args:
            instance (instance of TSP_Creator): problem to solve
        """
        self.boost = boost
        self.stop_after_secs = timeStop
        if(q0 != None):
            self.q0 = q0
        self.instance = instance
        self.n = instance.nPoints
        self.dist_mat = instance.dist_matrix
        _, self.L_nn = nn(instance.dist_matrix,
                          starting_node=np.random.choice(self.n))
        self.tau0 = 1./(float(self.n) * self.L_nn)

        # position collector for the Ants, TO BE UPDATED during the steps
        self.position = {i: None for i in range(ACS.m)}
        self.tour = {i: []
                     for i in range(ACS.m)}  # tour collector for the Ants
        #self.pheromone = {r: [self.tau0]*ACS.cl for r in range(self.n)}
        self.pheromone = {r: [self.tau0]*(self.n) for r in range(self.n)}
        self.candidate_list = {r: ACS.take_candidates(r, instance.dist_matrix) for r in range(
            self.n)}  # per tutte le citta', prende le citta vicine
        #self.eta = {r: [1/self.dist_mat[r, s] for s in ACS.take_candidates(r, instance.dist_matrix)] for r in range(self.n)}
        self.eta = {r: [1./(self.dist_mat[r, s]+np.finfo(np.double).eps)
                        for s in range(self.n)] for r in range(self.n)}
        for k in self.eta:
            self.eta[k][k] = 0.
        self.tour_len = {i: 0. for i in range(ACS.m)}
        self.best_tour = []
        self.best_ant = 0
        self.best_tour_len = 10e8
        self.iterations = 1000000
        self.tour_len_global = []
        self.current_iteration = 0
        self.tour_len_over_iters = []
        self.count_exploitation = 0
        self.count_exploration = 0

    def global_update(self):
        """Global pheromone updating
        """
        for k1, val1 in self.pheromone.items():

            for k2, val2 in self.pheromone.items():
                #print(self.pheromone[k1][k2], "pheromone before")
                if(k1 == k2):
                    self.pheromone[k1][k2] = 0.
                else:
                    delta_tau = 0.
                    if(k1 in self.best_tour and k2 in self.best_tour):
                        delta_tau = 1./self.best_tour_len

                    self.pheromone[k1][k2] = (
                        1. - self.alpha) * self.pheromone[k1][k2] + (self.alpha * delta_tau)
                    self.pheromone[k2][k1] = (
                        1. - self.alpha) * self.pheromone[k2][k1] + (self.alpha * delta_tau)
                    #print(self.pheromone[k1][k2], "pheromone after")

    def solve(self):
        """Solving ACS
        """
        threeMinutes = Thread(target=lambda: sleep(self.stop_after_secs))
        threeMinutes.start()
        for iteration in range(self.iterations):
            self.current_iteration = iteration
            # if(iteration % 5 == 0):
            #    print("iteration : ", iteration)
            self.loop()
            best_tour_ant = min(self.tour_len, key=self.tour_len.get)
            best_len = min(self.tour_len.values())
            self.tour_len_over_iters.append(best_len)
            self.tour_len_global.append(np.mean(list(self.tour_len.values())))
            if(best_len < self.best_tour_len):
                self.best_tour_len = best_len
                self.best_ant = best_tour_ant
                self.best_tour = self.tour[best_tour_ant]

            # print("best tour len this time: ",
            #     best_len, "by ant # ", best_tour_ant)
            if(self.boost):
                self.two_Opt_Improvement()
            self.global_update()

            if not threeMinutes.is_alive():
                break
            self.tour = {i: []
                         for i in range(ACS.m)}
            self.position = {i: None for i in range(self.m)}
            self.tour_len = {i: 0. for i in range(self.m)}

    def local_update(self, k):
        """local pheromone update

        Args:
            k (int): ant number
        """
        # print(self.tour)
        old_city = self.tour[k][-2]
        next_city = self.position[k]
        old_ph = self.pheromone[old_city][next_city]
        pheromon = ((1.-self.rho) * old_ph) + (self.rho * self.tau0)
        self.pheromone[old_city][next_city] = pheromon
        self.pheromone[next_city][old_city] = pheromon
        self.tour_len[k] += self.dist_mat[old_city, next_city]

        '''if(k==0):
      
      print("old ph ", old_ph)
      print(f"pherormone edges{(old_city, next_city)}")
      print("new pheromone", pheromon)#self.pheromone[old_city][next_city]'''

    def loop(self, verbose=False):
        """each ant contructs its own tour

        Args:
            verbose (bool, optional): log or not. Defaults to False.
        """

        for node_idx in range(self.n):
            # if(node_idx % 15 == 0):
            #    print(f"iteration #{node_idx}")
            if(node_idx < self.n-1):

                for k in range(self.m):
                    if(self.position[k] == None):
                        self.position[k] = np.random.randint(
                            low=0, high=self.n-1)
                        """if(k == 0 and verbose):
                            print(
                                f"ant {k} starting from position: {self.position[k]}")"""
                        self.tour[k].append(self.position[k])
                    q = np.random.random()

                    eta_beta = np.power(self.eta[self.position[k]], self.beta)
                    
                    s_list = eta_beta*np.array(self.pheromone[self.position[k]])
                    s_list_cities=np.argsort(s_list)
                    #print(s_list)
                    #print(s_list_cities)
                    #s_list=sorted(s_list)
                    #s_dict = []
                    
                    
                    
                    '''
                    for s, city in zip(s_list, self.candidate_list[self.position[k]]):
                        s_dict[city] = s

                    for city in self.tour[k]:
                        s_dict.pop(city, None)
                    if(not s_dict):
                        
                        print("Finished candidate list")
                        print(len(self.tour[k]))
                        for s, city in zip(s_list, self.take_other_cities(self.position[k], self.dist_mat)):
                            if(city not in self.tour[k]):
                                s_dict[city] = s
                    # print(s_dict)
                    '''
                    
                    if(q < self.q0):
                        self.count_exploitation += 1
                        #next_city = max(s_dict, key=s_dict.get)
                        for i in range(self.n):
                            if(s_list_cities[self.n-1-i] not in self.tour[k]):
                                next_city=s_list_cities[self.n-1-i]
                                break
                            
                        self.position[k] = next_city
                        self.tour[k].append(next_city)

                    else:
                        '''
                        somma_prob = sum(s_dict.values()) + \
                            np.finfo(np.float64).eps
                        for city, s in s_dict.items():
                            s_dict[city] = (s/somma_prob)
                            if(s_dict[city] == 0.):
                                s_dict[city] = np.finfo(np.float64).eps
                        somma_prob = sum(s_dict.values()) + \
                            np.finfo(np.float64).eps
                        for city, s in s_dict.items():
                            s_dict[city] = (s/somma_prob)
                        '''
                        possible_cities =[]
                        new_s_list=[]
                        for i in range(self.n):
                            if(i not in self.tour[k]):
                                new_s_list.append(s_list[i])
                            
                                possible_cities.append(i)
                        sum_val = sum(new_s_list)
                        prob = new_s_list/sum_val
                        
                        next_city = random.choices(
                            possible_cities, weights=prob, k=1)[0]

                        
                        self.tour[k].append(next_city)
                        self.position[k] = next_city

                        self.count_exploration += 1

                    self.local_update(k)
            else:
                for k in range(self.m):
                    self.tour[k].append(self.tour[k][0])
                    self.tour_len[k] += self.dist_mat[self.tour[k]
                                                      [-2], self.tour[k][-1]]

            #print("position ant 0", self.position[0])

        #print("path lengths", self.tour_len)
        '''
        if len(self.tour[k][:-1]) > len(set(self.tour[k][-1])):
            print("some repetitions")
      '''
        # if(k==0):
        #  plot_tour(self.instance, self.tour[k], k)

    def two_Opt_Improvement(self):
        best_tour, best_tour_len = twoOpt_with_cl(
            self.best_tour, self.best_tour_len, ic.dist_matrix, self.candidate_list)
        if(best_tour_len < self.best_tour_len):
            #print(f"initial tour: {self.best_tour_len}. Improved by 2 opt: {best_tour_len}")
            self.best_tour, self.best_tour_len = best_tour, best_tour_len


In [29]:
%matplotlib 
import pandas as pd
seeds = [0, 42, 123]
q_s = [0.5,0.98, 1.]
instances = ["eil76.tsp", "ch130.tsp", "d198.tsp"]
results = pd.DataFrame(columns=['boost', 'best integer len', '# iterations', 'AVG integer len', 'STD ', 'Optimum', 'Rel error'])
counter = -1
results.to_csv("CSV/results2.csv")
for q in q_s:
    
    list_of_len=[]
    if(q==1.):
        q = q-(13./ic.nPoints)
    print(f"-------  Q0 = {q} ---------")
    for seed in seeds:
        print(f"-------  SEED = {seed} ---------")
        random.seed(seed)
        np.random.seed(seed)
        for boost in [False, True]:
            print(f"-------  Boost = {boost} ---------")
            
            
            results = {i: [] for i in instances}
            for instance in instances:
                print(f"-------  INSTANCE = {instance} ---------")
                ic = TSP_Instance_Creator("standard", instance)
                
                acs = ACS(ic, q, boost, 180)
                # print(acs.pheromone)
                # print(acs.candidate_list)
                acs.solve()
                results[instance].append(boost)
                results[instance].append(acs.best_tour_len)
                results[instance].append(acs.current_iteration)
                results[instance].append(np.mean(acs.tour_len_global))
                results[instance].append(np.std(acs.tour_len_global))
                results[instance].append(ic.best_sol)
                results[instance].append(((acs.best_tour_len-ic.best_sol)*100.)/ic.best_sol)
                list_of_len.append(np.mean(acs.tour_len_global))
                
                #plt.plot(list(range(acs.current_iteration+1)),acs.tour_len_global)
                #plt.show()
            counter+=1
            results = pd.DataFrame(results).T
            results.columns = ['boost', 'best integer len', '# iterations', 'AVG integer len', 'STD ', 'Optimum', 'Rel error']
            results.to_csv("CSV/results2.csv", mode='a', header=False)
            print(results)
                #print(
                #    f"absulute best lenght= {acs.best_tour_len}, by ant # {acs.best_ant}. Tour:\n {acs.best_tour}")
        #plt.figure(8,8)
        #plt.plot(acs.tour_len_global, list(range(10)))
        #plt.show()
    
#print(f"difference len ACS - NN = {acs.best_tour_len - acs.L_nn}")


Using matplotlib backend: MacOSX
-------  Q0 = 0.5 ---------
-------  SEED = 0 ---------
-------  Boost = False ---------
-------  INSTANCE = eil76.tsp ---------
-------  INSTANCE = ch130.tsp ---------
-------  INSTANCE = d198.tsp ---------
           boost best integer len # iterations AVG integer len         STD   \
eil76.tsp  False            803.0         1621     1160.495561    31.054223   
ch130.tsp  False          10603.0          392    15160.285496   469.506399   
d198.tsp   False          26378.0          123    34798.992742  1129.931433   

           Optimum  Rel error  
eil76.tsp    538.0  49.256506  
ch130.tsp   6110.0  73.535188  
d198.tsp   15780.0  67.160963  
-------  Boost = True ---------
-------  INSTANCE = eil76.tsp ---------
-------  INSTANCE = ch130.tsp ---------
-------  INSTANCE = d198.tsp ---------
          boost best integer len # iterations AVG integer len         STD   \
eil76.tsp  True            598.0         1254     1160.698406    31.284915   
ch130.t

In [ ]:
%matplotlib tk
instance = instances[0]
ic = TSP_Instance_Creator("standard", instance)
acs = ACS(ic, 0.8, boost=True, timeStop=10)

#print(acs.eta)
acs.solve()
print(acs.best_tour)
total= float(acs.count_exploitation)+float(acs.count_exploration)
print("exploitations= ", float(acs.count_exploitation)*100/total, " % ")

plt.plot(list(range(acs.current_iteration+1)),acs.tour_len_global, color="red")
#plt.plot([np.argmax(acs.tour_len_over_iters), np.argmin(acs.tour_len_over_iters)], [max(acs.tour_len_over_iters), min(acs.tour_len_over_iters)], color="blue")
plt.show()

In [ ]:
print(acs.best_tour)



In [ ]:
%matplotlib tk
plot_tour(acs.instance, acs.best_tour,acs.best_ant)

# test twoOpt_with_cl 

the implementation of 2opt with the candidate list has worst performances in term of quality but achieves improvements using fewer computation

In [ ]:
from time import time


ic = TSP_Instance_Creator("standard", 'fl1577.tsp')

initial_sol, initial_len = nn(ic.dist_matrix, starting_node=np.random.choice(ic.nPoints))
acs = ACS(ic)


In [ ]:
'''start = time()
tour, len_new = twoOpt_with_cl(initial_sol, initial_len, ic.dist_matrix, acs.candidate_list)
print(f' 2opt with candidate: initial len {initial_len}, final len {len_new} \n execution time: {time() - start}')
'''
start = time()
tour, len_new = twoOpt(initial_sol, initial_len, ic.dist_matrix)
print(f' 2opt: initial len {initial_len}, final len {len_new} \n execution time: {time() - start}')

In [ ]:


list_time = []
for _ in range(5):
  start = time()
  _ = twoOpt_with_cl(initial_sol, initial_len, ic.dist_matrix, acs.candidate_list)
  list_time.append(time()- start)

print(f"mean {np.mean(list_time)},  std {np.std(list_time)}" )

In [ ]:

list_time = []
for _ in range(5):
  start = time()
  _ = twoOpt(initial_sol, initial_len, ic.dist_matrix)
  list_time.append(time() - start)

print(f"mean {np.mean(list_time)},  std {np.std(list_time)}" )